Problem statement : Falcon 7B is a strong base model - and its ability to outperform other open-source models. This model is trained on 1500B tokens of Refinedweb enhanced curated corpura

https://huggingface.co/tiiuae/falcon-7b-instruct#:~:text=Falcon%2D7B%20is%20a%20strong,See%20the%20OpenLLM%20Leaderboard.





#Instruct fine tune falcon 7b model-7B

# Installing Dependencies

In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
#trl, transformers, accelerate, peft, einops, wandb
#model monitorning - wandb (weights and biases) - for how loss is reducing, access artifacts.

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install datasets bitsandbytes einops wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=3f0b85f92e23be1cccb3b17c131a8c1f89e43018db1674421d2ea8fcc8738950
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


# Importing dataset :

In [5]:
from datasets import load_dataset #apache2.0 license highly rated conversations #instruct dataset to finetune
dataset_name = "timdettmers/openassistant-guanaco" #subset of openassistant dataset, highly rated conversations
dataset = load_dataset(dataset_name, split = "train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [6]:
dataset #text attribute with 9846 rows
#once everything is done, falcon 7B sharded model

Dataset({
    features: ['text'],
    num_rows: 9846
})

#Loading the model - Specifying Quantization Configuration

In [7]:
#loading the model falcon 7B
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "ybelkada/falcon-7b-sharded-bf16" #specify model name and bits and bytes configuration, using qlora, load quantized model, quantization config specified

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
model.config.use_cache = False  #finetuning quantized version of the model


# Loading tokenizer

In [10]:
# loading tokenizer
tokenizer= AutoTokenizer.from_pretrained(model_name, trust_remote_code = True) #downloading tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Lora Configuration https://arxiv.org/abs/2305.14314

In [11]:
from peft import LoraConfig  #specify lora configuration file, consider all the linear layers in the transformer block for maximum performance

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64


In [12]:
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout =lora_dropout,
    r = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["query_key_value", #fine tuning query key value,
                      "dense",
                      "dense_h_to_4h",
                      "dense_4h_to_h"
                      ]
)

In [13]:
#export PYTORCH_CUDA_ALLOC_CONF="max_split_size_mb:100"


#loading the trainer


In [14]:

from transformers import TrainingArguments  #load in 4bit

output_dir= "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps  = logging_steps,
    max_grad_norm = max_grad_norm,
    max_steps = max_steps,
    warmup_ratio = warmup_ratio,
    group_by_length = True,
    lr_scheduler_type = lr_scheduler_type
)


In [15]:
#pass to the trainer supervised fine tuning, use sft trianer from trl library for fine tuning part, specify
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

Preprocessing the model by upcasting the layer norms in float32 for more stable training

In [16]:
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.float32)


In [ ]:
trainer.train() #invokes training process

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.611000
20,1.529100
30,1.389800
40,1.463800
50,1.451600
60,1.374100
70,1.366000
80,1.355200
90,1.368500
100,1.413000


Step,Training Loss
10,1.611000
20,1.529100
30,1.389800
40,1.463800
50,1.451600
60,1.374100
70,1.366000
80,1.355200
90,1.368500
100,1.413000
